In [1]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Input
from tensorflow.python.keras import applications
from tensorflow.python.keras.layers import AveragePooling2D, GlobalAveragePooling2D
from tensorflow.python.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger
from tensorflow.python.keras.applications.mobilenet import MobileNet
from tensorflow.python.keras.layers import Conv2D, Convolution2D, MaxPooling2D, ZeroPadding2D, BatchNormalization, Activation
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras import backend as K
import numpy as np
import pandas as pd
from tensorflow.python.keras import layers
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.python.keras.models import load_model

In [2]:
#parameters
classes = 12
image_size = 224
batch_size= 32
train_total = 3820
validation_total = 930

In [3]:
model_mobilenet = MobileNet(include_top=False, weights = 'imagenet',input_shape=(image_size,image_size,3), alpha=0.5)

In [4]:
x = model_mobilenet.output
x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(classes, activation='softmax')(x)

In [5]:
model = Model(inputs=model_mobilenet.input, outputs=x)

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 16)      432       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 16)      64        
_________________________________________________________________
conv1_relu (Activation)      (None, 112, 112, 16)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 16)      144       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 16)      64        
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 112, 112, 16)      0         
__________

In [7]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.2,
                                   rotation_range = 20,
                                   height_shift_range=0.2,
                                   width_shift_range=0.2,
                                   zoom_range=0.2,
                                   fill_mode='reflect',
                                   horizontal_flip=True,
                                   vertical_flip=True)
validation_datagen = ImageDataGenerator(rescale = 1./255)

In [8]:
train_generator = train_datagen.flow_from_directory(
                    'train',
                    target_size=(image_size,image_size),
                    batch_size=batch_size,
                    class_mode='categorical',
                    shuffle=True
                    )
validation_generator = validation_datagen.flow_from_directory(
                        'validation',
                        target_size=(image_size,image_size),
                        batch_size=batch_size,
                        class_mode='categorical')

Found 3820 images belonging to 12 classes.
Found 930 images belonging to 12 classes.


In [9]:
model.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(lr=1e-4),
             metrics=['accuracy'])

In [10]:
model.load_weights('mobilenet_from_scratch_save_best_final_2_96977.hdf5')

In [11]:
model.evaluate_generator(validation_generator, int(np.ceil(validation_total/batch_size)))

[0.12311724872999294, 0.96344086021505382]

In [12]:
# MODEL SMALL FOR BINARY CLASSIFICATION FOR CLASS 0 AND CLASS 6
#V2 - weights from previous model_1, fixed bottom, only fine tuning last layers

In [13]:
#parameters
classes_2 = 2
image_size_2 = 600
batch_size_2 = 8
train_total_2 = 739
validation_total_2 = 178

In [14]:
model_mobilenet_2 = MobileNet(include_top=False, weights = None,input_shape=(image_size_2,image_size_2,3), alpha=0.5)

In [15]:
x = model_mobilenet_2.output
x = Dropout(0.5)(x)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(classes_2, activation='softmax')(x)

In [16]:
model_2 = Model(inputs=model_mobilenet_2.input, outputs=x)

In [17]:
model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 600, 600, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 300, 300, 16)      432       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 300, 300, 16)      64        
_________________________________________________________________
conv1_relu (Activation)      (None, 300, 300, 16)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 300, 300, 16)      144       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 300, 300, 16)      64        
_________________________________________________________________
conv_dw_1_relu (Activation)  (None, 300, 300, 16)      0         
__________

In [18]:
for layer in model_mobilenet_2.layers:
    layer.trainable = True

In [19]:
train_datagen_2 = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.2,
                                   rotation_range = 20,
                                   height_shift_range=0.2,
                                   width_shift_range=0.2,
                                   zoom_range=0.2,
                                   fill_mode='reflect',
                                   horizontal_flip=True,
                                   vertical_flip=True)
validation_datagen_2 = ImageDataGenerator(rescale = 1./255)

In [20]:
train_generator_2 = train_datagen.flow_from_directory(
                    'train_2',
                    target_size=(image_size_2,image_size_2),
                    batch_size=batch_size_2,
                    class_mode='categorical',
                    shuffle=True
                    )
validation_generator_2 = validation_datagen.flow_from_directory(
                        'validation_2',
                        target_size=(image_size_2,image_size_2),
                        batch_size=batch_size_2,
                        class_mode='categorical')

Found 739 images belonging to 2 classes.
Found 178 images belonging to 2 classes.


In [21]:
model_2.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(lr=1e-4),
             metrics=['accuracy'])

In [22]:
callbacks = [ModelCheckpoint(filepath='10_mobilenet_from_scratch_save_best_model_2_v3.hdf5', verbose=1, save_best_only=True, save_weights_only=True),
ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1),
EarlyStopping(monitor='val_loss', patience=10, verbose=1),
CSVLogger('./10-metrics_model_2_v2.csv')]

In [23]:
# loading weights from model_1 to model_2


In [24]:
for i in range(len(model.layers)-2):
    layer_1 = model.layers[i]
    layer_2 = model_2.layers[i]
    layer_2.set_weights(layer_1.get_weights())

In [25]:
x = np.zeros((256,2))
x.shape

(256, 2)

In [26]:
y = np.zeros(2)

In [27]:
y[0]=model.layers[-1].get_weights()[1][0]
y[1]=model.layers[-1].get_weights()[1][6]

In [28]:
x[:,0]=model.layers[-1].get_weights()[0][:,0]
x[:,1]=model.layers[-1].get_weights()[0][:,6]

In [29]:
model_2.layers[-1].set_weights([x,y])

In [30]:
model_2.evaluate_generator(validation_generator_2, int(np.ceil(validation_total_2/batch_size_2)))

[0.45261911344662142, 0.8202247191011236]

In [31]:
validation_generator_2 = validation_datagen.flow_from_directory(
                        'validation_2',
                        target_size=(image_size_2,image_size_2),
                        batch_size=batch_size_2,
                        class_mode='categorical',
                        shuffle=False)

Found 178 images belonging to 2 classes.


In [32]:
history_2 = model_2.fit_generator(
                    train_generator_2,
                    steps_per_epoch = int(np.ceil(train_total_2/batch_size_2)),
                    epochs=100,
                    validation_data=validation_generator_2,
                    validation_steps= int(np.ceil(validation_total_2/batch_size_2)),
                    verbose=2,
                    callbacks=callbacks)

Epoch 1/100


KeyboardInterrupt: 

In [33]:
model_2.load_weights('10_mobilenet_from_scratch_save_best_model_2_v3.hdf5')

In [34]:
model_2.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(lr=1e-4),
             metrics=['accuracy'])

In [97]:
history_2 = model_2.fit_generator(
                    train_generator_2,
                    steps_per_epoch = int(np.ceil(train_total_2/batch_size_2)),
                    epochs=100,
                    validation_data=validation_generator_2,
                    validation_steps= int(np.ceil(validation_total_2/batch_size_2)),
                    verbose=2,
                    callbacks=callbacks)

Epoch 1/100
Epoch 00000: val_loss did not improve
21s - loss: 0.3105 - acc: 0.8787 - val_loss: 0.3154 - val_acc: 0.8820
Epoch 2/100
Epoch 00001: val_loss improved from 0.24675 to 0.23316, saving model to 10_mobilenet_from_scratch_save_best_model_2_v2.hdf5
21s - loss: 0.3080 - acc: 0.8771 - val_loss: 0.2332 - val_acc: 0.8933
Epoch 3/100
Epoch 00002: val_loss did not improve
15s - loss: 0.3026 - acc: 0.8745 - val_loss: 0.2804 - val_acc: 0.8764
Epoch 4/100
Epoch 00003: val_loss did not improve
17s - loss: 0.2705 - acc: 0.8865 - val_loss: 0.2830 - val_acc: 0.8933
Epoch 5/100
Epoch 00004: val_loss did not improve
15s - loss: 0.4147 - acc: 0.8425 - val_loss: 0.2850 - val_acc: 0.8876
Epoch 6/100
Epoch 00005: val_loss did not improve
17s - loss: 0.2910 - acc: 0.8732 - val_loss: 0.2808 - val_acc: 0.8820
Epoch 7/100
Epoch 00006: val_loss did not improve
16s - loss: 0.2714 - acc: 0.8797 - val_loss: 0.3314 - val_acc: 0.8820
Epoch 8/100
Epoch 00007: val_loss improved from 0.23316 to 0.19558, saving

In [35]:
callbacks = [ModelCheckpoint(filepath='10_mobilenet_from_scratch_save_best_model_2_v3_temp.hdf5', verbose=1, save_best_only=True, save_weights_only=True),
ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1),
EarlyStopping(monitor='val_loss', patience=10, verbose=1),
CSVLogger('./10-metrics_model_2_v2.csv')]

In [36]:
model_2.load_weights('10_mobilenet_from_scratch_save_best_model_2_v3.hdf5')

In [100]:
model_2.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(lr=1e-5),
             metrics=['accuracy'])

In [101]:
history_2 = model_2.fit_generator(
                    train_generator_2,
                    steps_per_epoch = int(np.ceil(train_total_2/batch_size_2)),
                    epochs=100,
                    validation_data=validation_generator_2,
                    validation_steps= int(np.ceil(validation_total_2/batch_size_2)),
                    verbose=2,
                    callbacks=callbacks)

Epoch 1/100
Epoch 00000: val_loss improved from inf to 0.28032, saving model to 10_mobilenet_from_scratch_save_best_model_2_v2_temp.hdf5
24s - loss: 0.2570 - acc: 0.9087 - val_loss: 0.2803 - val_acc: 0.8764
Epoch 2/100
Epoch 00001: val_loss improved from 0.28032 to 0.22758, saving model to 10_mobilenet_from_scratch_save_best_model_2_v2_temp.hdf5
23s - loss: 0.2004 - acc: 0.9191 - val_loss: 0.2276 - val_acc: 0.9270
Epoch 3/100
Epoch 00002: val_loss improved from 0.22758 to 0.19548, saving model to 10_mobilenet_from_scratch_save_best_model_2_v2_temp.hdf5
22s - loss: 0.2401 - acc: 0.8970 - val_loss: 0.1955 - val_acc: 0.9101
Epoch 4/100
Epoch 00003: val_loss did not improve
21s - loss: 0.2041 - acc: 0.9087 - val_loss: 0.2615 - val_acc: 0.9045
Epoch 5/100
Epoch 00004: val_loss did not improve
20s - loss: 0.2867 - acc: 0.8862 - val_loss: 0.2840 - val_acc: 0.8820
Epoch 6/100
Epoch 00005: val_loss did not improve
21s - loss: 0.2385 - acc: 0.8953 - val_loss: 0.2637 - val_acc: 0.8933
Epoch 7/100

In [102]:
model_2.load_weights('10_mobilenet_from_scratch_save_best_model_2_v3_temp.hdf5')

In [37]:
model_2.evaluate_generator(validation_generator_2, int(np.ceil(validation_total_2/batch_size_2)))

[0.19622431861974424, 0.93258426966292129]

In [38]:
validation_generator_2 = validation_datagen.flow_from_directory(
                        'validation_2',
                        target_size=(image_size_2,image_size_2),
                        batch_size=batch_size_2,
                        class_mode='categorical',
                        shuffle=False)

Found 178 images belonging to 2 classes.


In [39]:
predictions_valid_2 = model_2.predict_generator(validation_generator_2,int(np.ceil(validation_total_2/batch_size_2)))
predictions_valid_2 = np.argmax(predictions_valid_2, axis=1)


In [40]:
len(predictions_valid_2)

178

In [41]:
true_positive_2 = np.zeros(2)
false_positive_2 = np.zeros(2)
true_negative_2 = np.zeros(2)
false_negative_2 = np.zeros(2)
for i in range(len(predictions_valid_2)):
    if(predictions_valid_2[i]==validation_generator_2.classes[i]):
        true_positive_2[predictions_valid_2[i]] = true_positive_2[predictions_valid_2[i]] + 1
    else:
        false_positive_2[predictions_valid_2[i]] = false_positive_2[predictions_valid_2[i]] + 1
        false_negative_2[validation_generator_2.classes[i]] = false_negative_2[validation_generator_2.classes[i]] +1

In [42]:
print('true_positive_2')
print(true_positive_2)
print('false_positive_2')
print(false_positive_2)
print('false_negative_2')
print(false_negative_2)
print('Total_2')
print(true_positive_2+false_negative_2)

true_positive_2
[  37.  129.]
false_positive_2
[  2.  10.]
false_negative_2
[ 10.   2.]
Total_2
[  47.  131.]


In [43]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory(
                    'test',
                    shuffle=False,
                    target_size=(image_size,image_size),
                    batch_size=batch_size,
                    class_mode=None)


Found 794 images belonging to 1 classes.


In [44]:
predictions = model.predict_generator(test_generator,int(np.ceil(794/batch_size)))
class_ids = {train_generator.class_indices[x]: x for x in train_generator.class_indices}
predicted_classes = [class_ids[x] for x in np.argmax(predictions, axis=1)]


In [45]:
num_class_0 = 0
num_class_6 = 0
for i in np.argmax(predictions, axis=1):
    if(i==0):
        num_class_0 = num_class_0 +1
    elif(i==6):
        num_class_6 = num_class_6 +1
        

In [46]:
indexes = []
for i in range(len(np.argmax(predictions, axis=1))):
    if((np.argmax(predictions, axis=1)[i]==0) or (np.argmax(predictions, axis=1)[i]==6)):
        indexes.append(i)

In [47]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory(
                    'test',
                    shuffle=False,
                    target_size=(image_size_2,image_size_2),
                    batch_size=batch_size,
                    class_mode=None)


Found 794 images belonging to 1 classes.


In [48]:
class_ids_2

NameError: name 'class_ids_2' is not defined

In [49]:
predictions_2 = model_2.predict_generator(test_generator,int(np.ceil(794/batch_size)))
class_ids_2 = {train_generator_2.class_indices[x]: x for x in train_generator_2.class_indices}
predicted_classes_2 = [class_ids_2[x] for x in np.argmax(predictions_2, axis=1)]


In [50]:
predicted_classes_2

['Black-grass',
 'Loose Silky-bent',
 'Black-grass',
 'Black-grass',
 'Loose Silky-bent',
 'Black-grass',
 'Loose Silky-bent',
 'Black-grass',
 'Black-grass',
 'Loose Silky-bent',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Loose Silky-bent',
 'Loose Silky-bent',
 'Black-grass',
 'Black-grass',
 'Loose Silky-bent',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Loose Silky-bent',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Loose Silky-bent',
 'Black-grass',
 'Loose Silky-bent',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Loose Silky-bent',
 'Loose Silky-bent',
 'Black-grass',
 'Black-grass',
 'Loose Silky-bent',
 'Black-grass',
 'Black-grass',
 'Loose Silky-bent',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Black-grass',
 'Loose Silky-bent',
 'Loose Silky-bent',
 'Black-grass',
 'Loose Silky-bent',
 'Loose Silky-bent',
 'Black-grass'

In [51]:
jana_sum = 0
for i in range(len(indexes)):
    if(predicted_classes[indexes[i]] != predicted_classes_2[indexes[i]]):
        jana_sum = jana_sum +1
        
jana_sum

16

In [52]:
for i in range(len(indexes)):
    predicted_classes[indexes[i]] = predicted_classes_2[indexes[i]]

In [53]:
num_class_0 = 0
num_class_6 = 0
for i in predicted_classes:
    if(i=='Black-grass'):
        num_class_0 = num_class_0 +1
    elif(i=='Loose Silky-bent'):
        num_class_6 = num_class_6 +1
        

In [54]:
num_class_0

35

In [55]:
num_class_6

122

In [56]:
from os.path import basename
test_ids = [basename(x) for x in test_generator.filenames]
submission = pd.DataFrame({'file':test_ids,'species':predicted_classes})
submission.to_csv('submission_mobilenet_two_models_v3.csv', encoding="utf8", index=False)

In [57]:
from IPython.display import FileLink
FileLink('submission_mobilenet_two_models_v3.csv')

/home/janardhan/plant_seeding/submission_mobilenet_two_models_v3.csv